<a href="https://colab.research.google.com/github/fatemafaria142/Comprehensive-Exploration-of-Large-Language-Models-in-Mental-Health-Advice-Generation/blob/main/Mental_Health_Advice_Generation_using_TinyLlama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Install Required Packages**

In [1]:
!pip install accelerate peft bitsandbytes transformers trl datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 30.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.9/78.9 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.6 MB/s eta 0:00:00


# **Load the required packages**

In [2]:
import torch
from datasets import load_dataset, Dataset
from peft import LoraConfig, AutoPeftModelForCausalLM
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments
from trl import SFTTrainer
import os

#### **Dataset Link:** https://huggingface.co/datasets/Amod/mental_health_counseling_conversations

In [5]:
dataset="Amod/mental_health_counseling_conversations"
model_id="TinyLlama/TinyLlama-1.1B-Chat-v1.0"
output_model="tinyllama-Instructionse-v1"

# **Dataset preparation**

In [6]:
def prepare_train_data(data_id, num_samples=3200):
    data = load_dataset(data_id, split="train")

    # Take the first 3200 rows
    data = data.select(range(min(num_samples, len(data))))
    instructions_template = "Put yourself in the shoes of a thoughtful mental health counselor. Imagine you're all set to help with kindness and understanding, ready to offer care and support. Your task is to extend understanding and guidance, providing support to individuals dealing with mental health challenges."
    data_df = data.to_pandas()
    data_df["text"] = data_df[["Context","Response"]].apply(lambda x: "Instructions:\n" + instructions_template + "  \n User\n" + x["Context"] + "\nCounselor's Response\n"+ x["Response"], axis=1)
    data = Dataset.from_pandas(data_df)

    return data

In [7]:
data = prepare_train_data(dataset)

In [8]:
data

Dataset({
    features: ['Context', 'Response', 'text'],
    num_rows: 3200
})

In [9]:
data[0]

{'Context': "I'm going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here.\n   I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it.\n   How can I change my feeling of being worthless to everyone?",
 'Response': "If everyone thinks you're worthless, then maybe you need to find new people to hang out with.Seriously, the social context in which a person lives is a big influence in self-esteem.Otherwise, you can go round and round trying to understand why you're not worthless, then go back to the same crowd and be knocked down again.There are many inspirational messages you can find in social media. \xa0Maybe read some of the ones which state that no person is worthless, and that everyone has a good purpose to their life.Also, since our culture is so saturated with the belief that if someone doesn't feel good about themselves that this is someh

In [10]:
data[1]

{'Context': "I'm going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here.\n   I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it.\n   How can I change my feeling of being worthless to everyone?",
 'Response': 'Hello, and thank you for your question and seeking advice on this. Feelings of worthlessness is unfortunately common. In fact, most people, if not all, have felt this to some degree at some point in their life. You are not alone.\xa0Changing our feelings is like changing our thoughts - it\'s hard to do. Our minds are so amazing that the minute you change your thought another one can be right there to take it\'s place. Without your permission, another thought can just pop in there. The new thought may feel worse than the last one! My guess is that you have tried several things to improve this on your own even before reaching out on h

In [25]:
data[2]

{'Context': "I'm going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here.\n   I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it.\n   How can I change my feeling of being worthless to everyone?",
 'Response': "First thing I'd suggest is getting the sleep you need or it will impact how you think and feel. I'd look at finding what is going well in your life and what you can be grateful for. I believe everyone has talents and wants to find their purpose in life. I think you can figure it out with some help.",
 'text': "Instructions:\nPut yourself in the shoes of a thoughtful mental health counselor. Imagine you're all set to help with kindness and understanding, ready to offer care and support. Your task is to extend understanding and guidance, providing support to individuals dealing with mental health challenges.  \n User\nI'm going throug

In [26]:
data[3]

{'Context': "I'm going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here.\n   I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it.\n   How can I change my feeling of being worthless to everyone?",
 'Response': 'Therapy is essential for those that are feeling depressed and worthless. When I work with those that are experiencing concerns related to feeling of depression and issues with self esteem. I generally work with my client to help build coping skills to reduce level of depression and to assist with strengthening \xa0self esteem, by guiding my client with CBT practices. CBT helps with gaining a better awareness of how your thought process influences your\xa0belief system, and how your beliefs impact your actions and the outcome of your behaviors. \xa0This process isn’t easy but it helps teach an individual that we don’t always have con

In [32]:
data[300]

{'Context': 'I\'ve been having this ongoing problem for most of my life now. I am a young adult, and right now, driving and even being a passenger gives me terrible panic attacks and anxiety. I can\'t ride in the backseat or the front seat with the safety belt on. I have to lean the seat back a bit because the pressure on my back terrifies me. I miss so many opportunities to go places with friends and family because of my fears of having a panic attack. My family offers little to no support. They think it\'s fake or "all in my head." It makes me feel guilty and at times depressed to the point where I\'d rather never wake up anymore. I am not self-harming, I just feel meaningless. This constant fear has taken over my thoughts and my life. I was such a happy person. I want my life back. I want to travel with my friends and family again, but I don\'t know how. I\'ve almost given up hope on getting better and just accepting the fact that I\'ll live with this anxiety forever.',
 'Response':

## **We have to model the Model (not the base version)**

In [11]:
def get_model_and_tokenizer(mode_id):

    tokenizer = AutoTokenizer.from_pretrained(mode_id)
    tokenizer.pad_token = tokenizer.eos_token
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype="float16", bnb_4bit_use_double_quant=True
    )
    model = AutoModelForCausalLM.from_pretrained(
        mode_id, quantization_config=bnb_config, device_map="auto"
    )
    model.config.use_cache=False
    model.config.pretraining_tp=1
    return model, tokenizer

In [12]:
model, tokenizer = get_model_and_tokenizer(model_id)

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

# **Setting up the LoRA**

In [13]:
peft_config = LoraConfig(
        r=8, lora_alpha=16, lora_dropout=0.05, bias="none", task_type="CAUSAL_LM"
    )

# **Training Arguments**

In [15]:
training_arguments = TrainingArguments(
        output_dir=output_model,
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        optim="paged_adamw_32bit",
        learning_rate=2e-4,
        lr_scheduler_type="cosine",
        save_strategy="epoch",
        logging_steps=10,
        num_train_epochs=10,
        max_steps=350,
        fp16=True,
        # push_to_hub=True
    )

In [16]:
trainer = SFTTrainer(
        model=model,
        train_dataset=data,
        peft_config=peft_config,
        dataset_text_field="text",
        args=training_arguments,
        tokenizer=tokenizer,
        packing=False,
        max_seq_length=1024
    )

Map:   0%|          | 0/3200 [00:00<?, ? examples/s]

# **Training starts here**

In [17]:
trainer.train()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
10,2.543500
20,2.381400
30,2.176600
40,2.024200
50,1.890700
60,1.828000
70,1.918500
80,1.760200
90,1.849800
100,1.850300


TrainOutput(global_step=350, training_loss=1.8587199020385743, metrics={'train_runtime': 434.0797, 'train_samples_per_second': 3.225, 'train_steps_per_second': 0.806, 'total_flos': 3298426675937280.0, 'train_loss': 1.8587199020385743, 'epoch': 0.44})

# **Merging the LoRA with the base model**

In [18]:
from peft import AutoPeftModelForCausalLM, PeftModel
from transformers import AutoModelForCausalLM
import torch
import os

model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float16, load_in_8bit=False,
                                             device_map="auto",
                                             trust_remote_code=True)

model_path = "/content/tinyllama-Instructionse-v1/checkpoint-350"

peft_model = PeftModel.from_pretrained(model, model_path, from_transformers=True, device_map="auto")

model = peft_model.merge_and_unload()

In [19]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 2048)
    (layers): ModuleList(
      (0-21): 22 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=5632, bias=False)
          (up_proj): Linear(in_features=2048, out_features=5632, bias=False)
          (down_proj): Linear(in_features=5632, out_features=2048, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (lm_head)

# **Inference from the LLM for many examples**

In [20]:
def formatted_prompt(template, question)-> str:
    return f"<|im_start|>Instructions\n{template}<|im_end|>\n<|im_start|>User\n{question}<|im_end|>\n<|im_start|>Counselor's Response\n"

In [23]:
from transformers import GenerationConfig
from time import perf_counter

def generate_response(instruction, user_input):
    prompt = formatted_prompt(instruction, user_input)

    inputs = tokenizer([prompt], return_tensors="pt")
    generation_config = GenerationConfig(
        penalty_alpha=0.6,
        do_sample=True,
        top_k=5,
        temperature=0.5,
        repetition_penalty=1.2,
        max_new_tokens=1024,
        pad_token_id=tokenizer.eos_token_id
    )
    start_time = perf_counter()

    inputs = tokenizer(prompt, return_tensors="pt").to('cuda')

    outputs = model.generate(**inputs, generation_config=generation_config)
    decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()  # Remove leading and trailing whitespaces
    print(decoded_output)
    output_time = perf_counter() - start_time
    print(f"Time taken for inference: {round(output_time, 2)} seconds")

### **Example No:1**

In [24]:
instruction = "Put yourself in the shoes of a thoughtful mental health counselor. Imagine you're all set to help with kindness and understanding, ready to offer care and support. Your task is to extend understanding and guidance, providing support to individuals dealing with mental health challenges."
question = '''I'm going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here.\n   I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it. How can I change my feeling of being worthless to everyone?'''
generate_response(instruction=instruction, user_input=question)

<|im_start|>Instructions
Put yourself in the shoes of a thoughtful mental health counselor. Imagine you're all set to help with kindness and understanding, ready to offer care and support. Your task is to extend understanding and guidance, providing support to individuals dealing with mental health challenges.<|im_end|>
<|im_start|>User
I'm going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here.
   I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it. How can I change my feeling of being worthless to everyone?<|im_end|>
<|im_start|>Counselor's Response
It sounds like you have a lot on your plate right now. If you are able to take time for self-care activities such as exercise, meditation/yoga, reading, spending quality time with friends or family members that love you, then these will likely provide you with relief from your current distre

### **Example No:2**

In [30]:
instruction = "Put yourself in the shoes of a thoughtful mental health counselor. Imagine you're all set to help with kindness and understanding, ready to offer care and support. Your task is to extend understanding and guidance, providing support to individuals dealing with mental health challenges."
question = '''I self-harm, and I stop for awhile. Then when I see something sad or depressing, I automatically want to self-harm.'''
generate_response(instruction=instruction, user_input=question)

<|im_start|>Instructions
Put yourself in the shoes of a thoughtful mental health counselor. Imagine you're all set to help with kindness and understanding, ready to offer care and support. Your task is to extend understanding and guidance, providing support to individuals dealing with mental health challenges.<|im_end|>
<|im_start|>User
I self-harm, and I stop for awhile. Then when I see something sad or depressing, I automatically want to self-harm.<|im_end|>
<|im_start|>Counselor's Response
Hi there! It sounds like you are struggling with some very real issues that have led you to this place. Self-harm can be a coping mechanism but it does not solve any underlying problems. If your primary issue is feeling alone and isolated then maybe talking about those feelings would help. You could try reaching out to someone who may be able to provide more insight into what you might need. Also, if you feel comfortable sharing, consider seeking therapy. This will give you an opportunity to work 

### **Example No:3**

In [31]:
instruction = "Put yourself in the shoes of a thoughtful mental health counselor. Imagine you're all set to help with kindness and understanding, ready to offer care and support. Your task is to extend understanding and guidance, providing support to individuals dealing with mental health challenges."
question = '''There are many people willing to lovingly provide me with a home. I have food, clothes, and a university education, but I never feel like I belong. Even when I have a good time with people who are supposed to be close, I feel like I'm just out with friends and I never go home.'''
generate_response(instruction=instruction, user_input=question)

<|im_start|>Instructions
Put yourself in the shoes of a thoughtful mental health counselor. Imagine you're all set to help with kindness and understanding, ready to offer care and support. Your task is to extend understanding and guidance, providing support to individuals dealing with mental health challenges.<|im_end|>
<|im_start|>User
There are many people willing to lovingly provide me with a home. I have food, clothes, and a university education, but I never feel like I belong. Even when I have a good time with people who are supposed to be close, I feel like I'm just out with friends and I never go home.<|im_end|>
<|im_start|>Counselor's Response
I am so sorry that this has been happening for you. It sounds as though there may be some underlying issues that need addressing before you can move forward on your path towards belonging. If you would like further information about any specific issue(s) or if you want someone to talk to you about them, please contact me at 1-800-394-2557

### **Example No:4**

In [33]:
instruction = "Put yourself in the shoes of a thoughtful mental health counselor. Imagine you're all set to help with kindness and understanding, ready to offer care and support. Your task is to extend understanding and guidance, providing support to individuals dealing with mental health challenges."
question = '''I've been having this ongoing problem for most of my life now. I am a young adult, and right now, driving and even being a passenger gives me terrible panic attacks and anxiety. I can't ride in the backseat or the front seat with the safety belt on. I have to lean the seat back a bit because the pressure on my back terrifies me. I miss so many opportunities to go places with friends and family because of my fears of having a panic attack. My family offers little to no support. They think it's fake or "all in my head." It makes me feel guilty and at times depressed to the point where I'd rather never wake up anymore. I am not self-harming, I just feel meaningless. This constant fear has taken over my thoughts and my life. I was such a happy person. I want my life back. I want to travel with my friends and family again, but I don't know how. I've almost given up hope on getting better and just accepting the fact that I'll live with this anxiety forever.'''
generate_response(instruction=instruction, user_input=question)

<|im_start|>Instructions
Put yourself in the shoes of a thoughtful mental health counselor. Imagine you're all set to help with kindness and understanding, ready to offer care and support. Your task is to extend understanding and guidance, providing support to individuals dealing with mental health challenges.<|im_end|>
<|im_start|>User
I've been having this ongoing problem for most of my life now. I am a young adult, and right now, driving and even being a passenger gives me terrible panic attacks and anxiety. I can't ride in the backseat or the front seat with the safety belt on. I have to lean the seat back a bit because the pressure on my back terrifies me. I miss so many opportunities to go places with friends and family because of my fears of having a panic attack. My family offers little to no support. They think it's fake or "all in my head." It makes me feel guilty and at times depressed to the point where I'd rather never wake up anymore. I am not self-harming, I just feel me